In [1]:
import torch
from torchvision import datasets as ds

In [2]:
MNIST_ds = ds.MNIST("Training_Data/", train = True, download = True)


In [3]:
from sklearn.linear_model import LinearRegression
from pandas import DataFrame

In [4]:
def featureExtract(xValues,yValues):
    operation = LinearRegression()
    operation.fit(xValues,yValues)
    result1 = 2 * operation.intercept_ / (1 + operation.intercept_**2)
    result2 = (1 - operation.intercept_**2) / (1 + operation.intercept_**2)
    return result1, result2

def splitImage(image):
    L = image.numpy()
    B = [[] for _ in range(16)]
    for i in range (4):
        for j in range(4):
            for k in range(int(len(L[0])/4)):
                B[i*4 + j].append(L[int(len(L[0])/4)*i + k][int(len(L[0])/4)*j:int(len(L[0])/4)*(j+1)].tolist())
    return B

def processItem(item):
    numberOfBlackPixels = 0
    X,Y = [],[]
    for i in range(len(item)):
        for j in range(len(item)):
            if item[i][j] != 0:
                numberOfBlackPixels += 1
                X.append(i)
                Y.append(j)
    if len(X) > 0:
        df = DataFrame(X, columns=['xvalues'])
        df['yvalues'] = Y
        result2, result3 = featureExtract(df[['xvalues']], df.yvalues)
    else:
        result2, result3 = 0.0, 0.0
    return [numberOfBlackPixels/(len(item))**2, result2, result3]

In [7]:
splited_data = []
for i in range(60000):
    splited_data.append(splitImage(MNIST_ds.data[i]))
splited_data[0]

[[[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 3, 18],
  [0, 30, 36, 94, 154, 170, 253]],
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [18, 18, 126, 136, 175, 26, 166],
  [253, 253, 253, 253, 225, 172, 253]],
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [255, 247, 127, 0, 0, 0, 0],
  [242, 195, 64, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0]],
 [[49, 238, 253, 253, 253, 253, 253],
  [18, 219, 253, 253, 253, 253, 253]

In [ ]:
train_data = []
for i in range(60000):
    features = []
    for item in splited_data[i]:
        features.append(processItem(item))
    train_data.append((torch.tensor(features), MNIST_ds[i][1]))
    print("Sample n°",i," done!")
train_data[0]

In [ ]:
import torch.nn as nn
import torch.nn.functional as fn

In [ ]:
inputSize = 3*16
outputSize = 10
class NumberDet(nn.Module):

    def __init__(self):
        super().__init__()
        self.input = nn.Linear(inputSize,48)
        self.hidden = nn.Linear(32,16)
        self.output = nn.Linear(16,outputSize)

    def forward(self,x):
        x = fn.relu(self.input(x))
        x = fn.relu(self.hidden(x))
        x = self.output(x)
        return fn.log_softmax(x, dim=1)